<a href="https://colab.research.google.com/github/kurokawa5/nishika/blob/main/tutorial_nishika_25_mansion_ver2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

不動産価格予測

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ご自身のパスを設定
YOUR_PATH = '/content/drive/MyDrive/05_workspace/06_nishika/01_predict_prices/'
#cd {YOUR_PATH}

In [ ]:
!pip install optuna xfeat japanize-matplotlib >> /dev/null

In [ ]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import missingno as msno
from time import time

sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

In [ ]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(float(moyori))
    return moyori

def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

In [ ]:
BASE_PATH = '/content/drive/MyDrive/05_workspace/06_nishika/01_predict_prices/'
paths = glob(BASE_PATH + 'input/train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(BASE_PATH + 'input/test.csv')
print("Dimensions of Train Dataset:" + str(train_df.shape))
print("Dimensions of Test Dataset:" + str(test_df.shape))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Dimensions of Train Dataset:(692882, 28)
Dimensions of Test Dataset:(22395, 27)


In [ ]:
train_df.head()

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,46040085,中古マンション等,NaN,46218,鹿児島県,霧島市,国分中央,国分(鹿児島),4,３ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2010年第2四半期,未改装,NaN,7.322219
1,46046952,中古マンション等,NaN,46201,鹿児島県,鹿児島市,平之町,鹿児島中央,15,NaN,...,NaN,NaN,NaN,商業地域,80.0,400.0,2015年第1四半期,未改装,NaN,6.255273
2,46043502,中古マンション等,NaN,46201,鹿児島県,鹿児島市,小松原,谷山(市電),8,２ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2015年第1四半期,未改装,NaN,6.919078
3,46056934,中古マンション等,NaN,46201,鹿児島県,鹿児島市,真砂本町,南鹿児島,10,３ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2013年第1四半期,未改装,NaN,7.397940
4,46043325,中古マンション等,NaN,46201,鹿児島県,鹿児島市,高麗町,鹿児島中央,9,１Ｋ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2015年第3四半期,改装済,NaN,6.518514


In [ ]:
test_df.head()

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,1000001,中古マンション等,NaN,1101,北海道,札幌市中央区,旭ケ丘,円山公園,25,４ＬＤＫ,...,住宅,NaN,NaN,NaN,第１種低層住居専用地域,40.0,80.0,2021年第1四半期,未改装,NaN
1,1000002,中古マンション等,NaN,1101,北海道,札幌市中央区,旭ケ丘,円山公園,20,４ＬＤＫ,...,住宅,NaN,NaN,NaN,近隣商業地域,80.0,200.0,2021年第1四半期,未改装,NaN
2,1000066,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,3,２ＬＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2021年第1四半期,未改装,NaN
3,1000119,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＬＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2021年第1四半期,未改装,NaN
4,1000120,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,4,２ＬＤＫ,...,住宅,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2021年第1四半期,未改装,NaN


In [ ]:
train_df.isnull().sum()

ID                   0
種類                   0
地域              692882
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                661
最寄駅：名称            2695
最寄駅：距離（分）        23121
間取り              24432
面積（㎡）                0
土地の形状           692882
間口              692882
延床面積（㎡）         692882
建築年              20074
建物の構造            17386
用途               62613
今後の利用目的         364738
前面道路：方位         692882
前面道路：種類         692882
前面道路：幅員（ｍ）      692882
都市計画             19425
建ぺい率（％）          23625
容積率（％）           23625
取引時点                 0
改装               64026
取引の事情等          674203
取引価格（総額）_log         0
dtype: int64

In [ ]:
sub_df = pd.read_csv(BASE_PATH + 'input/sample_submission.csv')
sub_df.head()

,ID,取引価格（総額）_log
0,1000001,0
1,1000002,0
2,1000066,0
3,1000119,0
4,1000120,0


In [ ]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []
test_df[TARGET] = 0
df = pd.concat([train_df, test_df])

In [ ]:
rm_cols += ['市区町村コード']
for i,v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [ ]:
test_df[TARGET] = 0
train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)
print("Dimensions of Train Dataset:" + str(train_df.shape))
print("Dimensions of Test Dataset:" + str(test_df.shape))

Dimensions of Train Dataset:(692882, 19)
Dimensions of Test Dataset:(22395, 19)


In [ ]:
df = pd.concat([train_df, test_df])
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(715277, 19)

In [ ]:
df.head()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,14224288,神奈川県,横浜市港北区,小机町,小机,NaN,３ＬＤＫ,75,平成14年,ＲＣ,住宅,NaN,第１種低層住居専用地域,50.0,80.0,2005年第3四半期,未改装,NaN,7.414973
1,13302842,東京都,中野区,上高田,新井薬師前,NaN,２ＤＫ,50,昭和46年,ＳＲＣ,住宅,NaN,第１種低層住居専用地域,60.0,150.0,2005年第3四半期,未改装,NaN,7.230449
2,14226365,神奈川県,横浜市港北区,新横浜,新横浜,NaN,２ＬＤＫ,45,平成17年,ＲＣ,住宅,NaN,商業地域,80.0,800.0,2005年第3四半期,NaN,NaN,7.397940
3,13099111,東京都,中央区,日本橋堀留町,小伝馬町,3,３ＬＤＫ,60,平成12年,ＳＲＣ,住宅,NaN,商業地域,80.0,600.0,2005年第3四半期,未改装,NaN,7.591065
4,14232167,神奈川県,横浜市港北区,大豆戸町,大倉山(神奈川),NaN,３ＬＤＫ,45,昭和49年,鉄骨造,住宅,NaN,第１種住居地域,60.0,200.0,2005年第3四半期,未改装,NaN,7.113943


In [ ]:
df['取引時点'].unique()

array(['2005年第3四半期', '2005年第4四半期', '2006年第1四半期', '2006年第2四半期',
       '2006年第3四半期', '2006年第4四半期', '2007年第1四半期', '2007年第2四半期',
       '2007年第3四半期', '2007年第4四半期', '2008年第1四半期', '2008年第2四半期',
       '2008年第3四半期', '2008年第4四半期', '2009年第1四半期', '2009年第2四半期',
       '2009年第3四半期', '2009年第4四半期', '2010年第1四半期', '2010年第2四半期',
       '2010年第3四半期', '2010年第4四半期', '2011年第1四半期', '2011年第2四半期',
       '2011年第3四半期', '2011年第4四半期', '2012年第1四半期', '2012年第2四半期',
       '2012年第3四半期', '2012年第4四半期', '2013年第1四半期', '2013年第2四半期',
       '2013年第3四半期', '2013年第4四半期', '2014年第1四半期', '2014年第2四半期',
       '2014年第3四半期', '2014年第4四半期', '2015年第1四半期', '2015年第2四半期',
       '2015年第3四半期', '2015年第4四半期', '2016年第1四半期', '2016年第2四半期',
       '2016年第3四半期', '2016年第4四半期', '2017年第1四半期', '2017年第2四半期',
       '2017年第3四半期', '2017年第4四半期', '2018年第1四半期', '2018年第2四半期',
       '2018年第3四半期', '2018年第4四半期', '2019年第1四半期', '2019年第2四半期',
       '2019年第3四半期', '2019年第4四半期', '2020年第1四半期', '2020年第2四半期',
       '2020年第3四半期', '2020年第4四半期', '2021年第1四半期', '2021年

In [ ]:
val_min_idx = min(df[df['取引時点'].str.contains('2020年第3四半期|2020年第4四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2021年第1四半期|2021年第2四半期', regex=True)].index)
val_min_idx, test_min_idx

(663186, 692882)

In [ ]:
#予測範囲の確認
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2020年第3四半期', '2020年第4四半期'}

特徴量生成

In [ ]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    #print(i, e)
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [ ]:
df.head()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc
0,14224288,神奈川県,横浜市港北区,小机町,小机,NaN,３ＬＤＫ,75,平成14年,ＲＣ,住宅,NaN,第１種低層住居専用地域,50.0,80.0,2005年第3四半期,未改装,NaN,7.414973,0
1,13302842,東京都,中野区,上高田,新井薬師前,NaN,２ＤＫ,50,昭和46年,ＳＲＣ,住宅,NaN,第１種低層住居専用地域,60.0,150.0,2005年第3四半期,未改装,NaN,7.230449,0
2,14226365,神奈川県,横浜市港北区,新横浜,新横浜,NaN,２ＬＤＫ,45,平成17年,ＲＣ,住宅,NaN,商業地域,80.0,800.0,2005年第3四半期,NaN,NaN,7.397940,0
3,13099111,東京都,中央区,日本橋堀留町,小伝馬町,3,３ＬＤＫ,60,平成12年,ＳＲＣ,住宅,NaN,商業地域,80.0,600.0,2005年第3四半期,未改装,NaN,7.591065,0
4,14232167,神奈川県,横浜市港北区,大豆戸町,大倉山(神奈川),NaN,３ＬＤＫ,45,昭和49年,鉄骨造,住宅,NaN,第１種住居地域,60.0,200.0,2005年第3四半期,未改装,NaN,7.113943,0


In [ ]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836439,
 '京都府': 7.155935848555253,
 '佐賀県': 7.009937258263509,
 '兵庫県': 7.157742870483196,
 '北海道': 6.965967793677859,
 '千葉県': 7.155825324964995,
 '和歌山県': 7.022690210815773,
 '埼玉県': 7.1697442713966355,
 '大分県': 6.911537826573079,
 '大阪府': 7.158352973821201,
 '奈良県': 7.056884610234688,
 '宮城県': 7.0395442538048805,
 '宮崎県': 6.958678653711235,
 '富山県': 7.0469200713870155,
 '山口県': 7.045342730349803,
 '山形県': 7.087990602814902,
 '山梨県': 6.848839888686708,
 '岐阜県': 7.078095913486453,
 '岡山県': 7.03809073492747,
 '岩手県': 6.925002679060599,
 '島根県': 7.165376214700312,
 '広島県': 7.093449021655064,
 '徳島県': 6.912051981629745,
 '愛媛県': 6.966433011596662,
 '愛知県': 7.107301969238601,
 '新潟県': 6.913835156235044,
 '東京都': 7.377929204641064,
 '栃木県': 6.950636958071011,
 '沖縄県': 7.160046604428336,
 '滋賀県': 7.16602695816839,
 '熊本県': 6.970182251843263,
 '石川県': 6.929363348877185,
 '神奈川県': 7.2581182672098175,
 '福井県': 6.979575815840195,
 '福岡県': 6.992264905042226,
 '福島県': 6.9743805791401785,
 '秋田県': 6.9413137471126

In [ ]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.head(2)

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,...,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,14224288,神奈川県,横浜市港北区,小机町,小机,NaN,３ＬＤＫ,75,平成14年,ＲＣ,...,NaN,第１種低層住居専用地域,50.0,80.0,2005年第3四半期,未改装,NaN,7.414973,0,0.0
1,13302842,東京都,中野区,上高田,新井薬師前,NaN,２ＤＫ,50,昭和46年,ＳＲＣ,...,NaN,第１種低層住居専用地域,60.0,150.0,2005年第3四半期,未改装,NaN,7.230449,0,0.0


In [ ]:
df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

In [ ]:
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前
0,14224288,NaN,75,2002.0,50.0,80.0,7.414973,0,0.0,16
1,13302842,NaN,50,1971.0,60.0,150.0,7.230449,0,0.0,16


In [ ]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,-1
1,1,1,1,1,1,1,0,-1,0,0,-1


In [ ]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(df)/100
num_comb_df.head(2)

,面積（㎡）容積率（％）_combi
0,60.0
1,75.0


In [ ]:
stop

NameError: ignored

In [ ]:
# aggregationのagg_methodsで使用
def max_min(x):
    return x.max()-x.min()

def q75_q25(x):
    return x.quantile(0.75) - x.quantile(0.25)

In [ ]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                        agg_methods=[['sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod', 'count', max_min, q75_q25],
                        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))
agg_dfs[0].head(2)

In [ ]:
feat_df = pd.concat([num_df,le_df,num_comb_df]+agg_dfs, axis=1)
print(feat_df.shape)

In [ ]:
feat_df.head()

In [ ]:
#feat_df.dtypes

In [ ]:
#feat_df.isnull().sum()

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(20,10))
axes[0][0].hist(feat_df["最寄駅：距離（分）"], bins=20)
axes[0][0].set_title("最寄駅：距離（分）")

axes[0][1].hist(feat_df["面積（㎡）"], bins=200)
axes[0][1].set_title("面積（㎡）")
axes[0][1].set_xlim(0, 250)

axes[1][0].hist(feat_df["建築年"], bins=20)
axes[1][0].set_title("建築年")

axes[1][1].hist(feat_df["取引価格（総額）_log"], bins=20)
axes[1][1].set_title("取引価格（総額）_log")

plt.show()

In [ ]:
from sklearn.impute import SimpleImputer
# モデルの定義
imp = SimpleImputer()

# トランスフォーム
result_mean = imp.fit_transform(feat_df)

feat_df_imputed = pd.DataFrame(result_mean, columns=feat_df.columns)
feat_df_imputed

In [ ]:
feat_df_imputed.isnull().sum()

In [ ]:
feat_df_imputed.describe()

In [ ]:
#histogram
sns.distplot(feat_df_imputed['取引価格（総額）_log']);

モデル構築

In [ ]:
train_df = feat_df_imputed.iloc[:val_min_idx, :]
val_df = feat_df_imputed.iloc[val_min_idx:test_min_idx, :]
test_df = feat_df_imputed.iloc[test_min_idx:, :]
print(train_df.shape, val_df.shape, test_df.shape)

In [ ]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [ ]:
feat_cols

In [ ]:
cat_cols = list(le_df.columns) + ['取引時点_enc']
cat_cols

In [ ]:
train_df.head()

In [ ]:
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [ ]:
#feat_df_imputed_ss.describe()

In [ ]:
SEED = 0

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 42,
    'max_depth': 7,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.95,
    'min_data_in_leaf': 2,
    'learning_rate': 0.1,
    "boosting": "gbdt",
    "lambda_l1": 0.1,
    "lambda_l2": 10,
    "verbosity": -1,
    "random_state": 42,
    "num_boost_round": 50000,
    "early_stopping_rounds": 100
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

model = lgb.train(
    params,
    train_data, 
    categorical_feature = cat_cols,
    valid_names = ['train', 'valid'],
    valid_sets =[train_data, val_data], 
    verbose_eval = 100,
)

val_pred = model.predict(val_x, num_iteration=model.best_iteration)
score = mean_absolute_error(val_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(val_x.index, val_pred, val_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

In [ ]:
lgb.plot_importance(model, figsize=(12,8), max_num_features=50, importance_type='gain')
plt.tight_layout()
plt.savefig(BASE_PATH + 'output/feature_importance.png')
plt.show()
plt.close()

In [ ]:
test_pred = model.predict(test_x, num_iteration=model.best_iteration)
test_df[TARGET] = test_pred

In [ ]:
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(BASE_PATH + 'output/test_submission_ss.csv', index=False)

In [ ]:
SEED = 42

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 42,
    'max_depth': 7,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.95,
    'min_data_in_leaf': 2,
    'learning_rate': 0.1,
    "boosting": "gbdt",
    "lambda_l1": 0.1,
    "lambda_l2": 10,
    "verbosity": -1,
    "random_state": 42,
    "num_boost_round": 50000,
    "early_stopping_rounds": 100
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

model = lgb.train(
    params,
    train_data, 
    categorical_feature = cat_cols,
    valid_names = ['train', 'valid'],
    valid_sets =[train_data, val_data], 
    verbose_eval = 100,
)

val_pred = model.predict(val_x, num_iteration=model.best_iteration)
score = mean_absolute_error(val_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(val_x.index, val_pred, val_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

In [ ]:
from sklearn.linear_model import  LinearRegression, Lasso, Ridge
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
model_lr = LinearRegression()
model_lr.fit(train_x, train_y)

model1 = Lasso()
model1.fit(train_x, train_y)

model2 = Ridge()
model2.fit(train_x, train_y)

pred_lr = model_lr.predict(val_x)
pred1 = model1.predict(val_x)
pred2 = model2.predict(val_x)

print("LinearRegression: ", mean_absolute_error(val_y, pred_lr))
print("Lasso: ", mean_absolute_error(val_y, pred1))
print("Ridge: ",mean_absolute_error(val_y, pred2))

plt.plot(model.predict(val_x), linestyle="solid", 
         color="red",label="lr")
plt.plot(model1.predict(val_x), linestyle="solid", 
         color="green", label="lasso")
plt.plot(model2.predict(val_x), linestyle="solid", 
         color="blue", label='ridge')
plt.title("LinearRegression, Lasso, Ridge")
plt.legend()
plt.show()

In [ ]:
stop

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [ ]:
train_set = lgb.Dataset(train_x, train_y)
valid_set = lgb.Dataset(val_x, val_y)

In [ ]:
params = {
    "objective" : "regression",
    "metric" : "mae",
    'learning_rate': 0.1,
}

In [ ]:
model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = [train_set, valid_set],
    num_boost_round = 1000
)

## アイデア1（ss = preprocessing.StandardScaler()）

In [ ]:
"""
from sklearn import preprocessing
ss = preprocessing.StandardScaler()

# mm.fit_transform(l)
# ValueError: Expected 2D array, got 1D array instead:

result_ss = ss.fit_transform(feat_df_imputed)
feat_df_imputed_ss = pd.DataFrame(result_ss, columns=feat_df_imputed.columns)
"""

In [ ]:
%matplotlib inline
 
 
# データ解析用ライブラリ
import pandas as pd
import numpy as np
 
# データ可視化用ライブラリ
import matplotlib.pyplot as plt
 
# Scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
 
# LightGBM
import lightgbm as lgb

In [ ]:
import xgboost as xgb

In [ ]:
train_data = xgb.DMatrix(train_x, label=train_y)
eval_data = xgb.DMatrix(val_x, label=val_y)

In [ ]:
xgb_params = {
    "objective": "reg:squarederror",
    'eval_metric': "mae"
    }
evals = [(train_data, 'train'), (eval_data, 'eval')]

gbm = xgb.train(
    xgb_params,
    train_data,
    num_boost_round=100,
    early_stopping_rounds=10,
    evals=evals,
    )


preds = gbm.predict(X_data)

In [ ]:
# 欠損値を含む列のデータタイプを確認
na_columns = feat_df.isnull().sum()[feat_df.isnull().sum()>0].index.tolist()
feat_df[na_columns].dtypes.sort_values()

In [ ]:
## HEAT MAP
f, ax = plt.subplots(figsize=(12, 9))
## train_dfのNumeric Variableに関する相関関係を求める
mat = feat_df.corr('pearson')
## Matrixは対称なので、Uppper Triangle部分をマスキングする
mask = np.triu(np.ones_like(mat, dtype=bool))
## 色の設定
cmap = sns.diverging_palette(230, 20, as_cmap=True)
## Heat Map生成
sns.heatmap(mat, mask=mask, cmap=cmap, vmax=1, center=0, annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
feat_df[['最寄駅：距離（分）','面積（㎡）','建築年', '取引価格（総額）_log']].corr()

In [ ]:
sns.heatmap(feat_df[['最寄駅：距離（分）','面積（㎡）','建築年', '取引価格（総額）_log']].corr())

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(20,10))
axes[0].scatter(feat_df["最寄駅：距離（分）"],feat_df["取引価格（総額）_log"], alpha=0.1, color="red")
axes[0].set_xlabel("Distance (in minutes)", fontsize=20) # (6)x軸ラベル
axes[0].set_ylabel("Transaction price (total)", fontsize=20) # (7)y軸ラベル
axes[1].scatter(feat_df["面積（㎡）"],feat_df["取引価格（総額）_log"], alpha=0.1, color="red")
axes[1].set_xlabel("Area (m2)", fontsize=20) # (6)x軸ラベル
axes[1].set_ylabel("Transaction price (total)", fontsize=20) # (7)y軸ラベル
axes[2].scatter(feat_df["建築年"],feat_df["取引価格（総額）_log"], alpha=0.1, color="red")
axes[2].set_xlabel("Year of construction", fontsize=20) # (6)x軸ラベル
axes[2].set_ylabel("Transaction price (total)", fontsize=20) # (7)y軸ラベル
plt.show()

In [ ]:
# ライブラリーのインポート
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#データを標準化
sc = StandardScaler()
sc.fit(train_x) #学習用データで標準化
train_x_std = sc.transform(train_x)
test_x_std = sc.transform(test_x)

In [ ]:
# ライブラリーのインポート
from sklearn.linear_model import LinearRegression

# スコア計算のためのライブラリ
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# モデルの学習
lr = LinearRegression()
lr.fit(train_x_std, train_y)

# 回帰　
pred_lr = lr.predict(test_x_std)

# 評価
# 平均絶対誤差(MAE)
mae_lr = mean_absolute_error(test_y, pred_lr)

print("MAE : %.3f" % mae_lr)

# 回帰係数
print("Coef = ", lr.coef_)
# 切片
print("Intercept =", lr.intercept_)

In [ ]:
stop


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
#print(len(train_x)) 
#-> 663186
#print(len(test_x))
#-> 22395

In [ ]:
#stop

In [ ]:
test_pred = model_lr.predict(test_x)
test_df[TARGET] = test_pred
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(BASE_PATH + 'output/test_submission_LR.csv', index=False)

In [ ]:
test_pred = model1.predict(test_x)
test_df[TARGET] = test_pred
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(BASE_PATH + 'output/test_submission_Lasso.csv', index=False)

In [ ]:
test_pred = model2.predict(test_x)
test_df[TARGET] = test_pred
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(BASE_PATH + 'output/test_submission_ridge.csv', index=False)

In [ ]:
stop

In [ ]:
model = lgb.LGBMRegressor(n_estimators=10000)

#各ラウンドでの予測精度などを格納する辞書
evals_result = {}

model.fit(train_x, train_y,
        eval_set=[(train_x, train_y),(val_x, val_y)],
        eval_metric='mae',
        early_stopping_rounds=10,
        callbacks=[lgb.callback.record_evaluation(evals_result)],
        )

print('best iteration:', model.best_iteration_)

y_pred = model.predict(val_x)

In [ ]:
mae = np.sqrt(mean_absolute_error(val_y, y_pred))
print('MAE:', round(mae,3))